This notebook will be use to explore the metadata of the Fishing Global Watch of a Earth Engine Snippet (GFW/GFF/V1/fishing_hours)

Metadata stands for data about data. In case of images, metadata means details about the image and its production. Some metadata is generated automatically by the capturing device. 



In [1]:
# Import packages
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Authenticate google earth engine
ee.Authenticate()

Enter verification code:  4/1AfgeXvumorzLzTL6s37UhDKL9gLFtiatHrH6hKY2VtABkhQ0551O8w9FHzM



Successfully saved authorization token.


In [4]:
# Initialize google earth engine 
ee.Initialize()

In [5]:
# read in the data on fishing hours from google earth engine
dataset = ee.ImageCollection('GFW/GFF/V1/fishing_hours')
# read in data on total vessel hours from google earth engine
dataset2 = ee.ImageCollection('GFW/GFF/V1/vessel_hours')

In [6]:
dataset

In [7]:
dataset2

In [8]:
# Create a filter
elnino_data = dataset.filter(ee.Filter.date('2015-01-01', '2016-01-01'))
